In [1]:
# Install geemap package
import subprocess
try:
    import geemap
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
import geemap

In [ ]:
#initialize google earth engine
import ee
try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize()

In [21]:
#load landsat images
l8 = ee.ImageCollection("LANDSAT/LC8_L1T_TOA");
l8 = l8.filterDate('2017-01-01', '2017-12-30')

In [22]:
crop_detection_train = ee.FeatureCollection('users/jpraveenkanna/crop_detection_train');


In [ ]:
#details of training data shape files
#crop_detection_train.getInfo()

In [6]:
#Visualize training shape files
Map = geemap.Map()
Map.addLayer(crop_detection_train)
Map.centerObject(crop_detection_train,zoom = 14)
Map

In [23]:
#Compute NDVI and create a qualityMosaic
#https://developers.google.com/earth-engine/tutorials/tutorial_api_06

addNDVI = lambda image: image.addBands(image.normalizedDifference(['B5', 'B4']).rename('NDVI'))

withNDVI = l8.map(addNDVI)
greenest = withNDVI.qualityMosaic('NDVI')

In [24]:
#Visualize composite image
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}
Map.addLayer(greenest, visParams, 'Greenest pixel composite')
Map.centerObject(crop_detection_train,zoom = 14)
Map

In [25]:
#create feature points
def featurePoints(fp):
  geom = fp.geometry()
  cropId = ee.Number.parse(fp.get('Crop_Id_Ne'))
  feildId = fp.get('Field_Id')
  feature = ee.Feature(geom, {'CID': cropId, 'Field_Id': feildId})
  return feature


points = crop_detection_train.map(lambda fp: featurePoints(fp))

In [10]:
type(points) 

ee.featurecollection.FeatureCollection

In [11]:
points.first().getInfo()

{'geometry': {'coordinates': [[[21.844921982872364, -28.72151184903625],
    [21.845729093344865, -28.722439491069196],
    [21.845796049302518, -28.722443396700214],
    [21.846567407342885, -28.72215252037768],
    [21.846883992516844, -28.721830178174113],
    [21.846888450181503, -28.721684769014146],
    [21.846772550898976, -28.721503980080097],
    [21.846491627038226, -28.721020501014326],
    [21.845903033276496, -28.720509441503516],
    [21.844921982872364, -28.72151184903625]]],
  'type': 'Polygon'},
 'id': '000000000000000008fc',
 'properties': {'CID': 4, 'Field_Id': 2106},
 'type': 'Feature'}

In [26]:
#Bands considered for training
bands = greenest.bandNames()
bands.getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI']

In [27]:
#Get training data
training = greenest.sampleRegions(**{
  'collection': points,
  'properties': ['CID'],
  'scale': 150 })
training.first().getInfo()

{'geometry': None,
 'id': '000000000000000008fc_0',
 'properties': {'B1': 0.11209850758314133,
  'B10': 300.1511535644531,
  'B11': 299.74615478515625,
  'B2': 0.09461502730846405,
  'B3': 0.09763026237487793,
  'B4': 0.0677940621972084,
  'B5': 0.522388756275177,
  'B6': 0.21602670848369598,
  'B7': 0.10103455185890198,
  'B8': 0.08824414014816284,
  'B9': 0.0012887632474303246,
  'BQA': 20512,
  'CID': 4,
  'NDVI': 0.7702608108520508},
 'type': 'Feature'}

In [28]:
# Train random forest classifier
trained = ee.Classifier.randomForest(**{'numberOfTrees':100}).train(**{
      "features": training, 
      "classProperty": 'CID', 
      "inputProperties": bands,

    })
trained.setOutputMode('PROBABILITY')

#trained.getInfo()

In [29]:
trained.confusionMatrix().getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 133, 0, 0, 1, 0, 0, 0, 2, 0],
 [0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 69, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 546, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 292, 0, 0, 1, 0],
 [0, 2, 0, 0, 1, 0, 206, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 271, 0, 0],
 [0, 0, 0, 0, 2, 0, 0, 0, 810, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 3, 81]]

In [30]:
#classify the image
classified = greenest.classify(trained)

In [31]:

Map.centerObject(crop_detection_train,zoom=14)
Map.addLayer(classified, {'min': 0, 'max': 10, 'palette': ['green','red']},
  'classification');
Map

In [32]:
# view result of a region
test1 = classified.reduceRegion(**{
  'reducer': ee.Reducer.mode(),
  'geometry': crop_detection_train.first().geometry(),
  'scale': 100,
  'maxPixels': 1e9
})
test1.getInfo()

{'classification': 4}

In [33]:
trained.confusionMatrix().accuracy().getInfo()

0.9938271604938271